In [1]:
#Import Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Load Dataset
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
# dataset preprocessing
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
# drop column - which column not required we can drop it
dataset= dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
#checking 0 and 1 counts in purchased column - also this is imbalanced Data 
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
#input and coutput columns
independent= dataset [['Age', 'EstimatedSalary','Gender_Male']]
dependent = dataset ['Purchased']

In [8]:
#input column size
independent.shape

(400, 3)

In [9]:
dependent.shape

(400,)

In [10]:
# train and test set split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(independent,dependent,test_size=1/3,random_state=0)

#Standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train= sc.fit_transform(x_train)
x_test= sc.transform(x_test)

In [12]:
# Model Creation
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

param_grid = {'splitter':['best','random'],'criterion':['gini','entropy','log_loss'],'max_features':['sqrt','log2',None]}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score

# fitting the model for grid search 
grid.fit(x_train, y_train) 


Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [13]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(x_test) 

In [14]:
print(grid.best_params_)
table=pd.DataFrame.from_dict(re)
table

{'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001291,0.001064,0.000528,0.001057,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.868752,0.833323,0.869709,0.844370,0.852659,0.014302,6
1,0.000484,0.000968,0.004498,0.003882,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.791752,0.870047,0.850543,0.814409,0.826499,0.830650,0.027357,14
2,0.003126,0.003087,0.000753,0.000927,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.812243,0.866968,0.775815,0.867097,0.867097,0.837844,0.037584,9
3,0.000405,0.000809,0.001612,0.003224,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.804764,0.849057,0.815056,0.867097,0.826499,0.832494,0.022719,13
4,0.002007,0.003109,0.004028,0.004214,gini,None,best,"{'criterion': 'gini', 'max_features': None, 's...",0.812243,0.868752,0.833323,0.870362,0.924528,0.861842,0.038271,4
5,0.002302,0.003403,0.004102,0.003240,gini,None,random,"{'criterion': 'gini', 'max_features': None, 's...",0.843811,0.793754,0.833323,0.943699,0.905069,0.863931,0.053520,3
6,0.001213,0.000378,0.001818,0.000604,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.813511,0.814409,0.851527,0.844370,0.825716,0.018601,15
7,0.001463,0.000247,0.001908,0.000459,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.765716,0.868752,0.833323,0.849057,0.867097,0.836789,0.037826,10
8,0.001363,0.001083,0.001077,0.001158,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.782557,0.793754,0.738451,0.906166,0.905069,0.825199,0.068216,16
9,0.002530,0.002318,0.002089,0.002932,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.826263,0.828959,0.718497,0.849057,0.787943,0.802144,0.046260,18


In [15]:
# here going to use confusion matrix instead of R2_Score
from sklearn.metrics import confusion_matrix
cm = confusion_matrix (y_test,grid_predictions)
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[77  8]
 [ 9 40]]


In [16]:
# Classification report to find recall, f1_score, precision, support
from sklearn.metrics import classification_report
clf_report =classification_report(y_test,grid_predictions)
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.90      0.91      0.90        85
           1       0.83      0.82      0.82        49

    accuracy                           0.87       134
   macro avg       0.86      0.86      0.86       134
weighted avg       0.87      0.87      0.87       134



In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}: 0.8728513338534315


In [18]:
#age_input=float(input("Age:"))
#salary_input=float(input("BMI:"))
#sex_male_input=int(input("Sex Male 0 or 1:"))
#classifier.predict([[age_input,salary_input,sex_male_input]])